## Librerias y tokenizer.

In [1]:
import pandas as pd
import numpy as np
import pickle
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer


In [ ]:

# Descargar recursos adicionales si no los tienes instalados
# nltk.download('punkt')
# nltk.download('stopwords')
nltk.download('wordnet')

stemmer = SnowballStemmer('spanish')
lemmatizer = WordNetLemmatizer()


def _tokenizer(s):
    stemmer = SnowballStemmer('spanish')
    lemmatizer = WordNetLemmatizer()
    s = nltk.word_tokenize(s)
    stop_words = set(stopwords.words('spanish'))
    s = [token.lower() for token in s if token.lower() not in stop_words]
    s = [stemmer.stem(item) for item in s]
    s = [lemmatizer.lemmatize(token) for token in s]
    return s

In [65]:
_tokenizer("word>stop>stem", "Hola, cómo estas? quiero estudiar computación en celex e inglées. Ayudame con cseuami. fisic")

['hol',
 'com',
 'quier',
 'estudi',
 'comput',
 'celex',
 'ingle',
 'ayudam',
 'cseuami',
 'fisic']

## Cargar modelo

### Variables.

In [162]:
datos_path = "data/textos_2023_08_0111_50_30.json"
modelo_path = "model_2023_08_0111_50_30_4.pickle"

In [163]:
# Es importante importar para usar el modelo
from tfidf import TFIDF

with open(modelo_path, "rb") as f:
    model5:TFIDF = pickle.load(f)

## Cargar datos

In [157]:
# df = pd.read_json('data/textos_2023_07_3000_48_04.json')
df = pd.read_json(datos_path)


print(len(df))
print(df.keys())


2061
Index(['document', 'metadata'], dtype='object')


### Analizar datos.

In [166]:
df.document.map(len).min()
urls = df["metadata"].apply(lambda x: x.get("url"))
keywords = df["metadata"].apply(lambda x: x.get("keywords"))

s_length = df['document'].str.len()
bool_mask = (s_length == s_length.min())
# print(df['document'][245])
print(urls[245])

# Calcular la longitud de cada texto y agregarla como una nueva columna
df['longitud'] = df['document'].apply(len)

# Calcular medidas estadísticas sobre la longitud de los textos
promedio_longitud = df['longitud'].mean()
mediana_longitud = df['longitud'].median()
maximo_longitud = df['longitud'].max()

# Imprimir los resultados

print("Información sobre el dataset:")
print("\nTamaño del dataset:",len(df))
print("\n\nLongitudes de los textos:")
# print(df)
print("\nMedidas estadísticas:")
print("Promedio:", promedio_longitud)
print("Mediana:", mediana_longitud)
print("Máximo:", maximo_longitud)

https://cbi.izt.uam.mx/index.php/secretaria-academica/dictamenes-de-concursos-de-oposicion
Información sobre el dataset:

Tamaño del dataset: 2061


Longitudes de los textos:

Medidas estadísticas:
Promedio: 4369.747210092189
Mediana: 2262.0
Máximo: 49423


In [168]:
urls[145]

'https://cbi.izt.uam.mx/coddaa/index.php/compu-pemc#plan-de-estudios'

## Consultar modelo.

In [165]:
query = "Plan de estudios en computación"
indices = model5.query_documents(query)
print(indices)
documents_most_similar = [df["metadata"].apply(lambda x: x.get("url"))[idx] for idx in indices]
print(documents_most_similar)
print(len(model5.get_vocabulary()))
model_stop_words = model5.get_stop_words()
print(list(model_stop_words)[:100])


{831: 0.1717383145640018, 1437: 0.1717383145640018, 1093: 0.1717383145640018, 1647: 0.1717383145640018, 1557: 0.1717383145640018, 1256: 0.1717383145640018, 1317: 0.1717383145640018, 1199: 0.1717383145640018, 4: 0.1712547665792782, 145: 0.1712547665792782}
['https://cbi.izt.uam.mx/coddaa/index.php/quim-pemc', 'https://cbi.izt.uam.mx/coddaa/index.php/quim-pemc#cbs', 'https://cbi.izt.uam.mx/coddaa/index.php/quim-pemc#csh', 'https://cbi.izt.uam.mx/coddaa/index.php/quim-pemc#cbi', 'https://cbi.izt.uam.mx/coddaa/index.php/quim-pemc#seleccion-de-optativas', 'https://cbi.izt.uam.mx/coddaa/index.php/quim-pemc#plan-de-estudios', 'https://cbi.izt.uam.mx/coddaa/index.php/quim-pemc#mapa-curricular', 'https://cbi.izt.uam.mx/coddaa/index.php/quim-pemc#optativas-extradivisionales', 'https://cbi.izt.uam.mx/coddaa/index.php/compu-pemc', 'https://cbi.izt.uam.mx/coddaa/index.php/compu-pemc#plan-de-estudios']
57528
Fueron eliminadas las stop words


TypeError: 'NoneType' object is not iterable

## Buscar página web

In [146]:
import re
urls = df["metadata"].apply(lambda x: x.get("url"))
keywords = df["metadata"].apply(lambda x: x.get("keywords"))
# patron = re.compile()
# if "https://www.cseuami.org/index.php/cambio-carrera" in df["metadata"].apply(lambda x: x.get("url")):
search_url = "https://cbi.izt.uam.mx/coddaa/index.php/fisica-pe"
try:
    index = [i for i, x in enumerate((urls.values==search_url)) if x][0]

    print(index)
    print(urls[index])
    print((df["document"][index]))
    print(keywords[index])
    print(_tokenizer(df["document"][index]))

except:
    print("No existe esa pagina web en los datos.")

2051
https://cbi.izt.uam.mx/coddaa/index.php/fisica-pe
Plan de Estudios y Mapa Curricular
Plan de Estudios y Mapa Curricular
Convocatorias
Plan de Estudios y Mapa Curricular
Plan de Estudios
Plan de Estudios
Mapa Curricular
Mapa Curricular
Licenciatura en Física
1
Universidad Autónoma Metropolitana - Unidad Iztapalapa

plan, estudio, mapa, curricular, convocatoria, licenciatura, física, universidad, autónoma, metropolitana, unidad, iztapalapa
['plan', 'estudi', 'map', 'curricul', 'plan', 'estudi', 'map', 'curricul', 'convocatori', 'plan', 'estudi', 'map', 'curricul', 'plan', 'estudi', 'plan', 'estudi', 'map', 'curricul', 'map', 'curricul', 'licenciatur', 'fisic', '1', 'univers', 'autonom', 'metropolitan', '-', 'unid', 'iztapalap']


## Probar extractor de páginas web.

In [92]:
    
obligatorios = ['uam']
# https://www.cseuami.org/index.php/cambio-carrera
incluidos = [
    '.izt',
    'uami',
    'iztapalapa'
]
excluidos = [
    'whatsapp',
    'facebook',
    'instagram',
    'twitter',
]

In [93]:
import validators
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import requests

In [90]:
def generador_patrones():
    # Obligatorios
    cadena = r"^"
    for element in obligatorios:
        cadena += (r"(?=.*" + element + r")")
        
    # Exclusión
    for element in excluidos:
        cadena += (r"(?!.*" + element + r")")
    
    # Opcionales
    cadena += r"(?=.*(?:"
    for element in incluidos:
        cadena += (element+r"|")
    return re.compile(cadena[:-1]+r")).*$")

In [43]:
pattern = re.compile(generador_patrones())
result = bool(pattern.match("https://cbi.izt.uam.mx/coddaa/index.php/mate-pemc"))
print(result)

True


In [123]:
urls = set()
base_url = "https://cbi.izt.uam.mx/coddaa/index.php/fisica-pe" # "https://www.cseuami.org/index.php/cambio-carrera"
req = requests.get(base_url)
soup = BeautifulSoup(req.content, 'html.parser')
patron = generador_patrones()
pattern = re.compile(patron)

for link in soup.find_all('a', href=True):       
    href = link.get('href')
    parsed_href = urlparse(href)
    # Verificar si el enlace es relativo
    if not parsed_href.netloc:
        # print(href) 
        url = urljoin(base_url, href)
    else:
        url = href
    if bool(pattern.match(url)) and not url.endswith('.jpg') and not url.endswith('.png') and not url.endswith('.jpeg'):
        urls.add(url)
print(urls)
print(len(urls))
                

{'https://cbi.izt.uam.mx/coddaa/index.php/compu-desc', 'https://cbi.izt.uam.mx/coddaa/index.php/fisica-da', 'https://cbi.izt.uam.mx/coddaa/index.php/descripcion', 'https://cbi.izt.uam.mx/coddaa/index.php/iq-desc', 'https://cbi.izt.uam.mx/coddaa/index.php/fisica-asp', 'https://www.cseuami.org/index.php/proceso', 'https://cbi.izt.uam.mx/coddaa/index.php/ceremonias/egresados-enero-2023', 'https://cbi.izt.uam.mx/coddaa/index.php/ih-desc', 'https://cbi.izt.uam.mx/coddaa/index.php/ceremonias/entrega-de-reconocimientos-al-alumnado-regular', 'https://cbi.izt.uam.mx/coddaa/index.php/posgrado/energia-y-medio-ambiente', 'https://cbi.izt.uam.mx/coddaa/index.php/eventos/carlos-graef-2023', 'https://cbi.izt.uam.mx/coddaa/index.php/fisica-pe#plan-de-estudios', 'https://cbi.izt.uam.mx/coddaa/index.php/eventos/carlos-graef', 'https://cbi.izt.uam.mx/coddaa/index.php/ceremonias', 'https://iquizayan.uam.mx:8443/sae/izt/aewbf001.omuestraframes?mod=1', 'https://cbi.izt.uam.mx/coddaa/index.php/vinculacion/mo

In [69]:
print(len(urls),urls)
new_urls = urls - set(['https://iquizayan.uam.mx:8443/sae/izt/aewbf001.omuestraframes?mod=1', 'ejlsjfel'])
print(len(new_urls), new_urls)

48 {'https://cbi.izt.uam.mx/coddaa/index.php/eventos/carlos-graef', 'https://www.cseuami.org/index.php/proceso', 'https://cbi.izt.uam.mx/coddaa/index.php/descripcion', 'https://cbi.izt.uam.mx/coddaa/index.php/ie-desc', 'https://cbi.izt.uam.mx/coddaa/index.php/compu-pemc', 'https://cbi.izt.uam.mx/coddaa/index.php/ceremonias/entrega-de-reconocimientos-al-alumnado-regular', 'https://cbi.izt.uam.mx/coddaa/index.php/nuevo-ingreso', 'https://cbi.izt.uam.mx/coddaa/index.php/fisica-desc', 'https://cbi.izt.uam.mx/coddaa/index.php/vinculacion/movilidad-estudiantil', 'https://cbi.izt.uam.mx/coddaa/index.php/catm-desc', 'https://cbi.izt.uam.mx/coddaa/index.php/eventos/carlos-graef-2023', 'https://cbi.izt.uam.mx/coddaa/index.php/mate-desc', 'https://cbi.izt.uam.mx/coddaa/index.php/quim-desc', 'https://cbi.izt.uam.mx/coddaa/index.php/eventos/carlos-graef-2022', 'https://cbi.izt.uam.mx/coddaa/index.php/compu-mt', 'https://cbi.izt.uam.mx/coddaa/index.php/posgrado/matematicas-aplicadas-e-industriales',

In [59]:
urls = set()
tmp_patron = generador_patrones()
pattern = re.compile(tmp_patron)

for link in soup.find_all('a', href=True):   
        
    href = link.get('href')
    if validators.url(href):
        parsed_href = urlparse(href)
        # Verificar si el enlace es relativo
        if not parsed_href.netloc: 
            url = urljoin(base_url, href)
        else:
            url = href
        # if url == "https://cbi.izt.uam.mx/coddaa/index.php/compu-pemc#plan-de-estudios":
        #     print("\n\nPAGINA ENCONTRADA\n\n")
        #      raise Exception("PAGINA ENCONTRADA")
        if bool(pattern.match(url)) and not url.endswith('.jpg') and not url.endswith('.png') and not url.endswith('.jpeg'):
            urls.add(url)
print(base_url, base_url=="https://cbi.izt.uam.mx/coddaa/index.php/compu-desc")
if base_url=="https://cbi.izt.uam.mx/coddaa/index.php/compu-desc":
    print(urls)
    raise("PAGINAS:::::")



https://cbi.izt.uam.mx/coddaa/index.php/compu-pemc False
